# Converting Rosanne's magnification files to magnified light curves and with the PLASTICC/SNANA format

### USER

In [18]:
# Dir of magnification files (those provided by Rosanne Di Stefano)
DirLensFiles = "/Users/arturo/Documents/Research/LSST/Rosanne/\
2018_07_16/1_Magnification/"

# Dir of simulated LSST star catalog
DirStarCatalog = "/Users/arturo/Documents/Research/LSST/Catalog/Subsamples/\
Region_ra_0.0_360.0_dec_-90.0_40.0/"

# Dir of expected lensing events in LSST catalog.
DirExpectedEvents = "/Users/arturo/Dropbox/Research/Articulos/14_PLASTICC/OGLE/\
Count_events_stars_size_10.0/Region_ra_0.0_360.0_dec_-90.0_40.0/"

# Dir save output
# DirSaveOutput = "/Users/arturo/Documents/Research/LSST/Rosanne/2018_07_16/2_output/"
DirSaveOutput = "/Users/arturo/Dropbox/Research/Articulos/14_PLASTICC/\
MockData/Samples/2018_07_16/2_output/"

# -------------------------------
# Given that javascript doesn't work in JupyterLab, I have to set the 
# name of the notebook by hand
NotebookName = 'Create_magnified_LCs.ipynb'

### Automatic

In [19]:
import numpy as np
import pandas as pd
import os # To use command line like instructions
import glob # To read the files in my directory
from matplotlib import pyplot as plt

5+6

11

In [20]:
#- Force the creation of the directory to save the plots.
#- "If the subdirectory does not exist then create it"
import os # To use command line like instructions
if not os.path.exists(DirSaveOutput+"2_plots"): os.makedirs(DirSaveOutput+"2_plots")

#### Get the name of this ipython notebook
To print it in the output text files as reference.

In [21]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "NotebookName = " + "'"+thename+".ipynb"+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [22]:
print(NotebookName)

# Given that javascript doesn't work in JupyterLab, I have to set the 
# name of the notebook by hand

Create_magnified_LCs.ipynb


In [23]:
# Get the current date and time
import datetime 

# Read the time and date now
now = datetime.datetime.now()

### Magnification & blending

In [24]:
# Function to convert from magnification to apparent magnitude

def Magnification2mag(Magf, mo):
        
    mag_int = mo - 2.5*np.log10(Magf) 
    return mag_int

print '# Test:'
print "#", Magnification2mag(2.14118954, 19.46)

# Test
# 18.6333622172
# 18.633362217176515

# Test:
# 18.6333622172


In [25]:
# Blending functions

def kappa_fun(mag_s, mag_2):
    'The kappa function'
    power_val = (mag_s - mag_2)/2.5
    return np.power(10.0, power_val)

#------------------------------------

def ff_fun(kappa):
    return 1.0/(kappa + 1.0)

#------------------------------------
# Observed total magnification from the 2 stars combined

def AA_total_blend_fun(AA, ff):
    return AA*ff + (1.0-ff)

#------------------------------------
# apparent magnitude from combined stars. No magnified

def appmag_base_fun(mag_s, kappa):
    return mag_s - 2.5*np.log10(1.0+kappa)

#------------------------------------
# apparent magnitude from combined stars. Magnified

def appmag_magnify_fun(mag_s, AA, kappa):
    return mag_s - 2.5*np.log10(AA+kappa)

## Read files

In [26]:
# Read the LSST star catalogue file with the magnitudes in ugrizY bands

starsdata_file = 'LSST_ra_0.0_360.0_dec_-90.0_40.0_Jump_100_sample.dat'

starsdata = np.genfromtxt(DirStarCatalog+starsdata_file,
                dtype=[float,float,float,float,float,
                       float,float,float]) 

print "#",starsdata_file
print "# %s stars data found in the file"%len(starsdata)

# LSST_ra_0.0_360.0_dec_-90.0_40.0_Jump_100_.dat
# 169229 stars data found in the file

# LSST_ra_0.0_360.0_dec_-90.0_40.0_Jump_100_sample.dat
# 4999 stars data found in the file


In [27]:
# Sample of the file
starsdata[:5]

array([ ( 249.2108339, -53.7600708,  16.21739,  14.93847,  14.53348,  14.35644,  14.19864,  14.09501),
       ( 262.8786919, -16.0070474,  18.41886,  17.15392,  16.62919,  16.38289,  16.20383,  16.10269),
       ( 267.8598182, -28.6188357,  27.47009,  24.50058,  22.16687,  20.72908,  19.54815,  18.84788),
       ( 254.9826929, -35.7618161,  19.72312,  17.90901,  16.87163,  16.30968,  15.86593,  15.61027),
       ( 242.4658103, -50.4776049,  30.9791 ,  30.8132 ,  27.20913,  24.40215,  22.17582,  20.8344 )], 
      dtype=[('f0', '<f8'), ('f1', '<f8'), ('f2', '<f8'), ('f3', '<f8'), ('f4', '<f8'), ('f5', '<f8'), ('f6', '<f8'), ('f7', '<f8')])

In [28]:
"""
array([ ( 269.9092004, -28.3269497,  20.99688,  19.00108,  17.79934,  17.13335,  16.60385,  16.29794),
       ( 267.0523731, -32.4916441,  28.21959,  25.46772,  22.80995,  21.13215,  19.76051,  18.95455),
       ( 273.7255786, -16.1384652,  29.85822,  28.62047,  25.17983,  22.84342,  21.00829,  19.91717),
       ( 276.9094877, -18.2496449,  22.17216,  20.27804,  19.11986,  18.48972,  18.00777,  17.73689),
       ( 273.004142 , -16.2747554,  29.37052,  27.52682,  24.3281 ,  22.22627,  20.55627,  19.57183)], 
      dtype=[('f0', '<f8'), ('f1', '<f8'), ('f2', '<f8'), ('f3', '<f8'), ('f4', '<f8'), ('f5', '<f8'), ('f6', '<f8'), ('f7', '<f8')])
"""
0

0

In [29]:
# Read the lensing metadata file:

lens_metadata_file = 'tau_dist_information_sample.txt'
lens_metadata = np.genfromtxt(DirLensFiles+lens_metadata_file,
                             dtype=[int,int,float,float,float,float])

print "#",lens_metadata_file
print "# %s LCs data found in the file. "%len(lens_metadata)

# tau_dist_information.txt
# 150000 LCs data found in the file.

# Sample of the array
lens_metadata[:5]

# tau_dist_information_sample.txt
# 10000 LCs data found in the file. 


array([(1, 475,   9.721,  0.3023 , -0.0199 ,   3.42 ),
       (2, 409,   4.222,  0.9684 , -0.03732,   1.365),
       (3, 480,   9.714,  0.04296,  0.07496,  23.178),
       (4, 469,  15.75 ,  0.4264 ,  0.1546 ,   2.502),
       (5, 277,  36.91 ,  1.344  ,  0.1781 ,   1.175)], 
      dtype=[('f0', '<i8'), ('f1', '<i8'), ('f2', '<f8'), ('f3', '<f8'), ('f4', '<f8'), ('f5', '<f8')])

In [30]:
"""
array([(1, 197,   48.48 ,  1.602  , -0.1665  ,   1.112),
       (2,  30,    3.692,  3.059  ,  0.1651  ,   1.015),
       (3,  84,  134.1  ,  2.219  ,  0.1174  ,   1.044),
       (4,  84,   40.01 ,  2.202  , -0.005246,   1.045),
       (5, 480,    3.228,  0.04616, -0.05398 ,  21.596)], 
      dtype=[('f0', '<i8'), ('f1', '<i8'), ('f2', '<f8'), ('f3', '<f8'), ('f4', '<f8'), ('f5', '<f8')])
"""
0

0

In [31]:
# Read the table containing the number of expected events per square degree
# inferred from OGLE sky map.

expectedEventsLSST_file = "Table_LSST_stars_events_interpol_left_.dat"
expectedEventsLSST_np = np.genfromtxt(DirExpectedEvents+expectedEventsLSST_file)

print len(expectedEventsLSST_np)

expectedEventsLSST_np[:5]

468


array([[   0.,   10.,  -90.,  -80.,  600.,    6.],
       [  10.,   20.,  -90.,  -80.,  700.,    7.],
       [  20.,   30.,  -90.,  -80.,  900.,    9.],
       [  30.,   40.,  -90.,  -80.,  600.,    6.],
       [  40.,   50.,  -90.,  -80.,  400.,    4.]])

In [32]:
"""
array([[   250.,    251.,    -40.,    -39.,   6700.,     69.],
       [   251.,    252.,    -40.,    -39.,  11100.,    114.],
       [   252.,    253.,    -40.,    -39.,  12200.,    131.],
       [   253.,    254.,    -40.,    -39.,  12000.,    123.],
       [   254.,    255.,    -40.,    -39.,   7700.,     79.]]) """
0

0

In [33]:
# Create a copy array of "expectedEventsLSST_np"
# This will be used to save the counting in each square

count_expectedEvents_np = np.copy(expectedEventsLSST_np)

print "%s boxes in this file."%len(count_expectedEvents_np)
count_expectedEvents_np[:5]

468 boxes in this file.


array([[   0.,   10.,  -90.,  -80.,  600.,    6.],
       [  10.,   20.,  -90.,  -80.,  700.,    7.],
       [  20.,   30.,  -90.,  -80.,  900.,    9.],
       [  30.,   40.,  -90.,  -80.,  600.,    6.],
       [  40.,   50.,  -90.,  -80.,  400.,    4.]])

In [34]:
# Reset to zero all the numbers of events in the last column

for i in range(len(count_expectedEvents_np)):               
    count_expectedEvents_np[i][5] = 0.0
    
#-------------------
count_expectedEvents_np[:5]

array([[   0.,   10.,  -90.,  -80.,  600.,    0.],
       [  10.,   20.,  -90.,  -80.,  700.,    0.],
       [  20.,   30.,  -90.,  -80.,  900.,    0.],
       [  30.,   40.,  -90.,  -80.,  600.,    0.],
       [  40.,   50.,  -90.,  -80.,  400.,    0.]])

### MAIN LOOP

In [56]:
# Initialize the timing of computation
now_start = datetime.datetime.now()
time_start = now_start.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
print "# Starting time of computing: %s"%time_start

# Starting time of computing: 2018-07-19 (yyyy-mm-dd); 13:39 hrs.


In [57]:
#    Some parameters and settings

debug = True

# Plot the output light curves?
plot_LightCurves = True

# Difference in u magnitude as a criterium to write down 
# that row in the text file. This is helpful to avoid writting a lot of
# rows with exactly the same values of the magnitudes in ugrizY bands.
diffmag_fix = 0.01

# Rounding the magnified app mag
if diffmag_fix == 0.01:
    round_appmag = 2
elif diffmag_fix == 0.001:
    round_appmag = 3

# Maximum percentage of time of a LC with magnitude larger
# than 16 mags
Porcentage_maxTime = 18.0 # percentage

# Maximum apparent magnitude. LSST saturates for 
# objects brighter than 16 mag.
maxMag = 16.0 # mag

# Limit of dimmest objects observed by LSST
max_dim_mag = 24.0  # mag

# Maxiumum percentaje of observations dimmer than the dimmest limit
# to be account and written in the PLASTICC file.
Percentage_obs_MaxDim = 10.0  # percentage

# Minimal number of observations (that passed all the cuts) in
# a given light curve to be considered and written in the
# PLASTICC file.
MinNumGoodObs = 10

#####################################################
#####################################################

# Reset to zero all the numbers of events in the last column

for i in range(len(count_expectedEvents_np)):               
    count_expectedEvents_np[i][5] = 0.0

#------------------------------------------

textfile_1 = open(DirSaveOutput+'2_Lensing_lc_PLASTICC_format_.dat', 'w')
textfile_2 = open(DirSaveOutput+'2_LSST_lensingEvents_RA_DEC_.dat', 'w')
textfile_3 = open(DirSaveOutput+'2_Report_.dat', 'w')

now = datetime.datetime.now() # Read the time and date right now
text_timenow = now.strftime("%m/%d/%Y")
text_Date   = 'COMMENT: Magnified light curves and file created \
by Arturo Avelino on %s\n'%text_timenow
text_script = '# Script used: %s \n'%NotebookName
text_line = '#'+'-'*30 + '\n'
    
textfile_1.write("SURVEY: LSST\n")
textfile_1.write("FILTERS: ugrizY\n")
textfile_1.write("MODEL: Microlensing by Rosanne Di Stefano\n")
textfile_1.write("MODEL_PARNAMES: tau_e,u_0\n")
textfile_1.write(text_Date)
# textfile_1.write()

#------------

textfile_2.write("# SURVEY: LSST\n")
textfile_2.write("# FILTERS: ugrizY\n")
textfile_2.write("# MODEL: Microlensing by Rosanne Di Stefano\n")
textfile_2.write("# MODEL_PARNAMES: tau_e,u_0\n")
textfile_2.write("# "+text_Date)
textfile_2.write(text_script)
textfile_2.write(text_line)
textfile_2.write("# RA       DEC \n")

#------------

textfile_3.write("# SURVEY: LSST\n")
textfile_3.write("# FILTERS: ugrizY\n")
textfile_3.write("# MODEL: Microlensing by Rosanne Di Stefano\n")
textfile_3.write("# MODEL_PARNAMES: tau_e,u_0\n")
textfile_3.write("# "+text_Date)
textfile_3.write(text_script)
textfile_3.write(text_line)

#------------

print "# Number of useful rows written in the text \
file for a given star,"
print "# it is, when there is a change in magnitude \
at least > %s mag."%diffmag_fix

# Reset variables.
initial_row = 0; count_lensFile = 0;
countEvent = 0; count_succesful_stars = 0;

# Loop over the stars 
for j in range(20):  # tmp  
# for j in range(len(starsdata)):
    
    ra  = starsdata['f0'][j]
    dec = starsdata['f1'][j]
    
    #####################################################

    # Loop over all the boxes.
    
    # Reset counter
    box_row_index = 0;
    
    # Find out in what box the current star is located, 
    # then count it for that box.
    for i4 in range(len(count_expectedEvents_np)):
        
        ra_min_int = count_expectedEvents_np[i4][0]
        ra_max_int = count_expectedEvents_np[i4][1]
        de_min_int = count_expectedEvents_np[i4][2]
        de_max_int = count_expectedEvents_np[i4][3]
        num_events_int = count_expectedEvents_np[i4][5]
        num_events_lim_int = expectedEventsLSST_np[i4][5]

        # Find out in what box the current star is located, 
        # then count it for that box.
        if (ra > ra_min_int and ra < ra_max_int and
            dec > de_min_int and dec < de_max_int):
            
            # Row in the array corresponding to the box where
            # the star is located. If this star passes the next
            # cutoffs then I'll update the counter for this box
            # in the 'expectedEventsLSST_np' array
            box_row_index = i4
            
            if debug:
                print "\n %s"%('-'*30)
                print "      Star %s in LSST catalog."%(j+1)
                print "Star found on box %s"%box_row_index
                print "ra = %.2f, dec = %.2f, count = %i (%i), \
(%.0f,%.0f,%.0f,%.0f)"%(ra, dec, num_events_int, 
            num_events_lim_int,
            ra_min_int, ra_max_int, de_min_int, de_max_int)
            
            # If I've already found the location of the star
            # in box, then break this loop and move on to the
            # rest of this script.
            break
    
    ##########################################################
    
    # Compute the total -maximum- magnification from the 
    # 2 stars in u-band.
    
    magStar_u_s = starsdata['f2'][j] # source star
    magStar_u_2 = starsdata['f2'][j+1] # 2nd star
    
    # Peak magnification of the source star:
    AA_at_peak_int = lens_metadata[count_lensFile][5]
    
    # Compute some intermediate values:
    kappa_int = kappa_fun(magStar_u_s, magStar_u_2)
    ff_int = ff_fun(kappa_int)
    
    # total -maximum- magnification
    AA_total_max_int = AA_total_blend_fun(AA_at_peak_int, ff_int)
    
    if debug:
        print "Magnification of: source star = %.3f , total = %.3f."%(
        AA_at_peak_int, AA_total_max_int)
    
    #------------------------------------------
    
    # If the maximum number of events in a given box has
    # not reached, then process it in the rest of this cell
    # script, otherwise discard it and move on to the next star.
    # Also, process the star if it has a change in magnification larger
    # than 1.1 (i.e., 10%).
    if (num_events_int <= num_events_lim_int and
        AA_total_max_int > 1.1):

        # Write a text file with the (ra,dec) of this event.
        textfile_2.write("%.4f  %.4f \n"%(ra,dec))
        
        #------------------------------------------

        # Number of observations in that light curve:
        number_rows = int(lens_metadata[count_lensFile][1])
        
        if debug:
            print "Number_rows = %s | count_lensFile = %s."%(
                number_rows, count_lensFile)

        #------------------------------------------
        # From the main magnification table, upload the information
        # of the magnification of an individual LC.
        # It is a very large file so I use pandas to read the datatable
        # by portions.
        lens_data = pd.read_table(DirLensFiles+'tau_dist_LCs_sample.dat', 
                          skiprows=initial_row, nrows=number_rows, sep='\s+')

        if debug:
            print "Initial_row = %s, number_rows = %s."%(
                initial_row, number_rows)
            print "First row of the light curve:", lens_data.values[0]

        ##########################################################
        
        # I use this loop just to count the number of rows and to
        # discard the magnified stars with magnitudes larger than
        # 16 mags for more than a given porcentage of time.
        
        # Loop over the magnification rows for a given LC in the 
        # lensing file, in u-band.

        # Reset the counter for the number of rows for this event
        # based on the change in magnitude in u band:
        countNROW = 0;
        count_AboveMaxMag = 0;
        count_belowMaxDim = 0;
        
        # kappa of the blending equations.
        kappa_int = kappa_fun(magStar_u_s, magStar_u_2)
        
        for ii in range(1,len(lens_data)):
            
            Magnific = lens_data.values[ii][2]
            # old. mag_u = Magnification2mag(Magnific, magStar_u)
            
            # magnified apparent magnitude from the 2 blended stars.
            mag_u = appmag_magnify_fun(magStar_u_s, Magnific, kappa_int) 

            # Compute the difference between the previous apparent
            # magnitude and the current value:
            if ii > 1:
                Magnific_prev = lens_data.values[int(ii-1)][2]
                # old. mag_u_prev = Magnification2mag(Magnific_prev, magStar_u)
                mag_u_prev = appmag_magnify_fun(magStar_u_s, Magnific_prev, kappa_int) 
                diffmag = abs(round(mag_u,round_appmag) - 
                              round(mag_u_prev,round_appmag))
            else: diffmag = 1

            if diffmag >= diffmag_fix: 
                countNROW += 1
                if mag_u < maxMag:
                    count_AboveMaxMag += 1
                if mag_u > max_dim_mag:
                    count_belowMaxDim += 1

        # Determine the percentage of MJD with mag larger than 
        # the magnitude threshold:
        porcentage_int1 = (float(count_AboveMaxMag)*100.)/(float(countNROW+2))

        # Determine the percentage of MJD with mag dimmer than 
        # the magnitude threshold:
        porcentage_dim_int1 = (float(count_belowMaxDim)*100.)/(float(countNROW+2))
        
        if debug:
            print "countNROW =  %s | porcentage_int1 = %.2f."%(
            countNROW, porcentage_int1)
            if countNROW < 1:
                print "Fail because the change in magnitude along the \
entire light curve is \nless than %s mag and/or because countNROW < 10 \
obs in the LC that passed this cut."%diffmag_fix
            if porcentage_int1 > Porcentage_maxTime:
                print "Fail because %.2f percent > %.2f percent limit."%(
                porcentage_int1, Porcentage_maxTime)

        #########################################

        # Consider the LCs that have less than the maximum 
        # porcentage of time of a LC with magnitude larger
        # than the threshold magnitude only.
        # Also, consider only LCs with at least a given 
        # number of observations.

        if (porcentage_int1 < Porcentage_maxTime and
            porcentage_dim_int1 < Percentage_obs_MaxDim and
            countNROW > MinNumGoodObs):

            countEvent = countEvent + 1

            textfile_1.write(" \n")
            textfile_1.write(text_line)
            textfile_1.write(" \n")
            textfile_1.write("START_EVENT: %s \n"%countEvent)

            # Write the metadata info for this star.
            textfile_1.write("NROW: %s  RA: %.5f   DEC: %.5f. \n"%(
                (countNROW+2), ra, dec))
            
            # Useful to print and keep track of the LC number in 
            # Rosanne metadata
            Rosanne_LC_num = lens_metadata[count_lensFile][0]

            if debug:
                print "-WRITING-: ROSANNE LC NUMBER: %s."%(Rosanne_LC_num)
                print "LC num = %s | Num of rows = %s \
| Num obs to write = %s."%(Rosanne_LC_num, 
                number_rows, (countNROW+2))
            
            #-----------------------------------------

            # Write the parameters of the specific event:

            textfile_1.write("PARVAL: %.4f  %.4f \n"%(
                lens_metadata[count_lensFile][2], 
                lens_metadata[count_lensFile][3]) )
            
            if debug:
                print "par1 = %.4f | par2 = %.4f."%(
                    lens_metadata[count_lensFile][2],
                    lens_metadata[count_lensFile][3])

            #------------------------------------------

            # Apparent magnitude star in different bands.
            # Source star
            magStar_u1 = starsdata['f2'][j]
            magStar_g1 = starsdata['f3'][j]
            magStar_r1 = starsdata['f4'][j]
            magStar_i1 = starsdata['f5'][j]
            magStar_z1 = starsdata['f6'][j]
            magStar_y1 = starsdata['f7'][j]

            # 2nd star = the next in the LSST catalog.
            magStar_u2 = starsdata['f2'][j+1]
            magStar_g2 = starsdata['f3'][j+1]
            magStar_r2 = starsdata['f4'][j+1]
            magStar_i2 = starsdata['f5'][j+1]
            magStar_z2 = starsdata['f6'][j+1]
            magStar_y2 = starsdata['f7'][j+1]
            
            # Compute kappa for the blended stars:
            kappa_u_int = kappa_fun(magStar_u1, magStar_u2)
            kappa_g_int = kappa_fun(magStar_g1, magStar_g2)
            kappa_r_int = kappa_fun(magStar_r1, magStar_r2)
            kappa_i_int = kappa_fun(magStar_i1, magStar_i2)
            kappa_z_int = kappa_fun(magStar_z1, magStar_z2)
            kappa_y_int = kappa_fun(magStar_y1, magStar_y2)
            
            # Compute the base app mag from the blended stars:
            magStar_u = appmag_base_fun(magStar_u1, kappa_u_int)
            magStar_g = appmag_base_fun(magStar_g1, kappa_g_int)
            magStar_r = appmag_base_fun(magStar_r1, kappa_r_int)
            magStar_i = appmag_base_fun(magStar_i1, kappa_i_int)
            magStar_z = appmag_base_fun(magStar_z1, kappa_z_int)
            magStar_y = appmag_base_fun(magStar_y1, kappa_y_int)

            #------------------------------------------
            
            # old. time_first = lens_data.values[0][1] # initial time
            time_first = lens_data.values[1][1] - 1.0 # initial time

            # Magnitudes of the template
            textfile_1.write("T: %9.3f  %.3f  %.3f  %.3f  %.3f  %.3f  \
%.3f \n"%(time_first, magStar_u, magStar_g, magStar_r, magStar_i,
                magStar_z, magStar_y))
            
            if debug: print "Initial time template = %.4f."%time_first

            # OLD:
            # Write the first event: it is equal in mags to template
            # textfile_1.write("S: %9.4f  %.3f  %.3f  %.3f  %.3f  %.3f  \
            # %.3f \n"%(
            #     time_first, magStar_u, magStar_g, magStar_r, magStar_i,
            #     magStar_z, magStar_y))

            #---------------------------------------------------
            # Loop over the magnification file for a given star

            if plot_LightCurves:
                time_list = []
                mag_u_list = []
                mag_g_list = []

                time_list  += [time_first]
                mag_u_list += [magStar_u]
                mag_g_list += [magStar_g]
            
            if debug: 
                print "len(lens_data) = %s."%len(lens_data)
                print "First row of values in 'lens_data' array:"
                print "%i | %.4f | %.3f."%(
                    lens_data.values[0][0],lens_data.values[0][1],
                    lens_data.values[0][2])
                
            # Reset dummy counter to track the values of i3 in the
            # loop below.
            count_int3 = 0;
            # Count the number of observations that pass
            # the 'diffmag >= diffmag_fix' cutoff
            countNROW_2 = 0;
            
            # Loop over the magnification data.
            for i3 in range(1,len(lens_data)):

                if debug and count_int3 < 1:
                    print "First rows of values read in this loop:"
                    print "%i | %.4f | %.3f | loop index i3 = %s."%(
                        lens_data.values[i3][0],lens_data.values[i3][1],
                        lens_data.values[i3][2], i3)
                     
                time_int = lens_data.values[i3][1]
                Magnific = lens_data.values[i3][2]
                
                # old:
                # mag_u = Magnification2mag(Magnific, magStar_u)
                # mag_g = Magnification2mag(Magnific, magStar_g)
                # mag_r = Magnification2mag(Magnific, magStar_r)
                # mag_i = Magnification2mag(Magnific, magStar_i)
                # mag_z = Magnification2mag(Magnific, magStar_z)
                # mag_y = Magnification2mag(Magnific, magStar_y)
                
                # Compute the magnified app mag from the blended stars:
                mag_u = appmag_magnify_fun(magStar_u1, Magnific, kappa_u_int)
                mag_g = appmag_magnify_fun(magStar_g1, Magnific, kappa_g_int)
                mag_r = appmag_magnify_fun(magStar_r1, Magnific, kappa_r_int)
                mag_i = appmag_magnify_fun(magStar_i1, Magnific, kappa_i_int)
                mag_z = appmag_magnify_fun(magStar_z1, Magnific, kappa_z_int)
                mag_y = appmag_magnify_fun(magStar_y1, Magnific, kappa_y_int)

                # Compute the difference between the previous magnitude 
                # and the current value, and print only the times
                # when the difference is larger than "diffmag_fix"
                if i3 > 1:
                    Magnific_prev = lens_data.values[int(i3-1)][2]
                    # old. mag_u_prev = Magnification2mag(Magnific_prev, magStar_u)
                    mag_u_prev = appmag_magnify_fun(magStar_u1, Magnific_prev, kappa_u_int)
                    diffmag = abs(round(mag_u,round_appmag) - 
                                  round(mag_u_prev,round_appmag))
                else: diffmag = 1

                if diffmag >= diffmag_fix:

                    text_05 = 'S: %9.3f  %.3f  %.3f  %.3f  %.3f  %.3f  %.3f \n'%(
                        time_int, mag_u, mag_g, mag_r, mag_i, mag_z, mag_y)
                    textfile_1.write(text_05)

                    # Count the number of observations that pass
                    # the 'diffmag >= diffmag_fix' cutoff
                    countNROW_2 += 1
                    
                    if debug and countNROW_2==1:
                        print "Num obs = %i | Second time = %.4f. | Magnific = %.3f."%(
                            lens_data.values[i3][0], time_int, Magnific)

                    if plot_LightCurves:
                        time_list  += [time_int]
                        mag_u_list += [mag_u] 
                        mag_g_list += [mag_g]  

                    # A hack to define the last MJD. I will use this to 
                    # write down the last event where the MJD will the the
                    # last used time + 1 day, and the magnitudes will
                    # be the same than the template.
                    time_int2 = time_int
                    
                # Dummy counter to track the very first value 
                # in this loop.
                count_int3 +=1 

            # Write the last event: it is equal in mags to template
            textfile_1.write("S: %9.3f  %.3f  %.3f  %.3f  %.3f  %.3f  \
%.3f \n"%((time_int2+1.0), magStar_u, magStar_g, magStar_r, magStar_i,
                magStar_z, magStar_y))        

            textfile_1.write("END_EVENT: %s \n"%countEvent)
            
            if plot_LightCurves:
                time_list  += [time_int2+1.0]
                mag_u_list += [magStar_u] 
                mag_g_list += [magStar_g]  
            
            # if debug:
            #     print "j = %s: countNROW = %s, countNROW_2 = %s."%(
            #         j, countNROW,countNROW_2)
                
            # Given that this star passed all the cuts then add one 
            # count to the box where the star is located:
            count_expectedEvents_np[box_row_index][5] += 1
            
            if debug:
                print "Box number (= row) %s: added one count."%box_row_index
                print "SUCCESSFULL star! %s = Rosanne light-curve number."%(
                    Rosanne_LC_num)

            #------------- PLOT THE LCs ------------------- 

            if plot_LightCurves:
                # Creating the plot 
                plt.figure()

                # Plot with points and lines (recommended)
                plt.plot(time_list, mag_u_list, lw=0.5, marker=".", ms=2) 

                # just the line:
                # plt.plot(time_list, mag_u_list, lw=2, ls = "-", color = 'blue') 
                # plt.plot(time_list, mag_g_list, lw=2, ls = "-", color = 'green') 

                plt.xlim(min(time_list)-5, max(time_list)+5)
                plt.ylim(max(mag_u_list)+0.1, min(mag_u_list)-0.1)

                plt.xlabel('time (days)')
                plt.ylabel('apparent magnitude')
                plt.title('u-band lensed and blended %s LSST star (%s Rosanne LC)'%(
                    j, Rosanne_LC_num))

                plt.savefig(DirSaveOutput+'2_plots/plot_%s_LSST_%s_Rosanne_%s.png'%(
                    countEvent, j, Rosanne_LC_num), dpi=120)
                plt.close()

        #--------------------------------------
        # Update the value of "initial_row"
        initial_row = initial_row + number_rows
        
        # Update counter to be use as index read the lines in the 
        # lensing metadata array.
        count_lensFile += 1

        if debug:
            print "Updated: initial_row = %s , count_lensFile = %s."%(
                initial_row,count_lensFile)
            
    
##################################################

# Write timing

text_16 = "\n\n#     Time Summary \n"
text_17 = "# Starting time of computing: %s \n"%time_start

textfile_3.write("# \n")
now_end = datetime.datetime.now()
time_end = now_end.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
text_18 = "# Ending time of computing: %s \n"%time_end

# Compute the time the  computations took:
time_compute = now_end - now_start
text_19 = "# Total time of computing:  %s \n"%time_compute

textfile_3.write(text_16);textfile_3.write(text_17);
textfile_3.write(text_18);textfile_3.write(text_19);

#---------------------------

text_22 = "\n# %s total succesful stars written in the PLASTICC file. \n"%countEvent
textfile_3.write(text_22)
#---------------------------
        
textfile_1.close(); textfile_2.close(); textfile_3.close();

print text_16,text_17, text_18, text_19, text_22

# Number of useful rows written in the text file for a given star,
# it is, when there is a change in magnitude at least > 0.01 mag.

 ------------------------------
      Star 1 in LSST catalog.
Star found on box 132
ra = 249.21, dec = -53.76, count = 0 (3453), (240,250,-60,-50)
Magnification of: source star = 3.420 , total = 3.138.
Number_rows = 475 | count_lensFile = 0.
Initial_row = 0, number_rows = 475.
First row of the light curve: [ 1.       -0.019899  1.      ]
countNROW = 64 | porcentage_int1 = 83.33.
Fail because 83.33 percent > 18.00 percent limit.
Updated: initial_row = 475 , count_lensFile = 1.

 ------------------------------
      Star 2 in LSST catalog.
Star found on box 278
ra = 262.88, dec = -16.01, count = 0 (4910), (260,270,-20,-10)
Magnification of: source star = 1.365 , total = 1.365.
Number_rows = 409 | count_lensFile = 1.
Initial_row = 475, number_rows = 409.
First row of the light curve: [ 1.       -0.037317  1.      ]
countNROW = 41 | porcentage_int1 = 0.00.
-

In [58]:
textfile_1.close();textfile_1.close();textfile_1.close();
textfile_1.close();textfile_1.close();textfile_1.close();

textfile_2.close();textfile_2.close();textfile_2.close();
textfile_2.close();textfile_2.close();textfile_2.close();

textfile_3.close();textfile_3.close();textfile_3.close();
textfile_3.close();textfile_3.close();textfile_3.close();

plt.close();plt.close();plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();plt.close();plt.close();
plt.close();plt.close();plt.close();plt.close();plt.close();

----------

# Scratch
